## 1.计算香农熵

In [1]:
import math

In [5]:
def calcShannonEnt(dataSet):
    numEntries = len(dataSet)
    labelCounts = {}
    for featVec in dataSet:
        currentLabel = featVec[-1] # 看后面以最后一列作为key
        labelCounts[currentLabel] = labelCounts.get(currentLabel,0) + 1
    shannonEnt = 0.0
    for key in labelCounts:
        prob = float(labelCounts[key])/numEntries # 计算概率
        shannonEnt -= prob * math.log(prob,2) # 这是计算公式，2为底的log函数与概率的乘积
    return shannonEnt

## 2.测试熵的计算

In [9]:
def createDataSet():
    dataSet = [
        [1,1,'yes'],
        [1,1,'yes'],
        [1,0,'no'],
        [0,1,'no'],
        [0,1,'no']
    ]
    labels = ['no surfacing','flippers']
    return dataSet,labels

In [10]:
data,labels = createDataSet()
print(calcShannonEnt(data))

0.9709505944546686


In [11]:
data[0][-1] = 'xxx'
print(calcShannonEnt(data))

1.3709505944546687


**数据越混乱，熵越大**

## 2.划分数据集

In [15]:
def splitDataSet(dataSet,axis,value):
    retDataSet = [] # 新建一个，因为传的是引用
    for row in dataSet:
        if row[axis] == value:
            # 下面的拆分去掉了axis这一项
            reducedRow = row[:axis]
            reducedRow.extend(row[axis+1:])
            retDataSet.append(reducedRow)
    return retDataSet

In [14]:
# 关于extend和append的区别
a = [1,2,3]
b = [4,5,6]
a.append(b)
print(a)
a.extend(b)
print(a)

[1, 2, 3, [4, 5, 6]]
[1, 2, 3, [4, 5, 6], 4, 5, 6]


## 4.测试划分

In [16]:
data,labels = createDataSet()
print(data)
print(splitDataSet(data,0,0))

[[1, 1, 'yes'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']]
[[1, 'no'], [1, 'no']]


## 5.选择最好的划分数据集

In [17]:
def chooseBestFetureToSplit(dataSet):
    numFetures = len(dataSet[0]) - 1
    baseEntroy = calcShannonEnt(dataSet)
    bestInfoGain = 0.0;bestFeture = -1
    for i in range(numFetures):
        fetureList = [example[i] for example in dataSet]
        uniqueVals = set(fetureList) # 唯一的分类标签列表
        newEntroy = 0.0
        for value in uniqueVals:
            subDataSet = splitDataSet(dataSet,i,value)
            prob = len(subDataSet)/float(len(dataSet))
            newEntroy += prob * calcShannonEnt(subDataSet) # 这是求香农熵的均值公式
        # 信息增益,变小才是有增益
        infoGain = baseEntroy - newEntroy 
        if infoGain > bestInfoGain:
            bestInfoGain = infoGain
            bestFeture = i
    return bestFeture

In [18]:
# 测试下
data,labels = createDataSet()
print(data)
print(chooseBestFetureToSplit(data))

[[1, 1, 'yes'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']]
0


**结果表示按第0个特征进行划分最好（总共就2个特征）**